## load model and data

In [1]:
from datasets import load_dataset

dataset = load_dataset("recastai/coyo-75k-augmented-captions")
sentences = dataset['train']['short_caption']
sentences = [sentence[0] for sentence in sentences]

Generating train split:   0%|          | 0/73480 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/935 [00:00<?, ? examples/s]

In [2]:
import pandas as pd
import torch
from transformers import CLIPProcessor, CLIPModel

device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
2024-06-08 14:25:37.014861: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 14:25:37.014959: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 14:25:37.150231: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one h

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

## create and save embeddings

In [3]:
batch_size=32
text_features = []
for i in range(0, len(sentences), batch_size):
    batch = sentences[i:i+batch_size]
    inputs = processor(text=batch, return_tensors="pt", padding=True, truncation=True)
    inputs = inputs.to(device)
    with torch.no_grad():
        batch_feature = model.get_text_features(**inputs)
    text_features.append(batch_feature.to("cpu"))
    del inputs,batch_feature
    torch.cuda.empty_cache()

In [4]:
text_features = torch.cat(text_features,dim=0)
text_features_np = text_features.numpy()
embeddings = pd.DataFrame(text_features_np)
embeddings['prompt'] = sentences
embeddings.to_csv("clip_embeddings.csv")